In [1]:
# Ref:
# YouTube Video
# Sam Witteveen
# Creating an AI Agent with LangGraph Llama 3 & Groq
# https://www.youtube.com/watch?v=lvQ96Ssesfk

In [2]:
!pip install -U -q google-generativeai # Install the Python SDK
#!pip -q install groq
!pip -q install tavily-python

In [3]:
!pip -q install -U sentence-transformers
!pip -q install faiss-gpu

In [4]:
!pip -q install PyPDF2
!pip -q install PyMuPDF # fitz

In [5]:
import pandas as pd
import numpy as np
import json
import os
import re
import google.generativeai as genai
#from google.colab import userdata

In [33]:
NUM_SEARCH_RESULTS = 3

# The embeddings and the dataframe created and saved in Part 1
PATH_TO_EMBEDS = '../input/part-1-build-an-arxiv-rag-search-system-w-faiss/compressed_array.npz'
PATH_TO_DF = '../input/part-1-build-an-arxiv-rag-search-system-w-faiss/compressed_dataframe.csv.gz'

# Define the API clients

In [7]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()


In [8]:
from tavily import TavilyClient

tavily_client = TavilyClient(api_key = user_secrets.get_secret("TAVILY_API_KEY"))

In [9]:
import google.generativeai as genai

genai.configure(api_key = user_secrets.get_secret("GOOGLE_API_KEY"))



# What is the objective?

- Create an llm research assistant that has a chat interface


# Create a list of agents

AGENTS
1. chat_agent
2. router_agent
3. research_agent
4. final_answer_agent

TOOLS
- Function that returns an avaerage dog weight given a dog breed
- Calculate function that returns answers to calculations

BLOCKS
- ReAct block

# Helper functions

In [10]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [11]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history



In [12]:
def initialize_message_history(system_message):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        }
                    ]

    return message_history



In [13]:
import requests
import fitz  # or import PyPDF2

def fitz_extract_pdf_text(pdf_url):

    # URL of the PDF file
    #url = "https://arxiv.org/pdf/your_arxiv_paper.pdf"
    response = requests.get(pdf_url, timeout=5)

    # Save the PDF file
    pdf_path = "arxiv_paper.pdf"
    with open(pdf_path, "wb") as file:
        file.write(response.content)

    # Open and parse the PDF file
    document = fitz.open(pdf_path)  # or use PyPDF2

    full_text = ""

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text = page.get_text("text")
        #print(f"Page {page_num + 1}:\n{text}\n")
                            
        full_text = full_text + f"Page {page_num + 1}:\n{text}\n\n"
                            
    return full_text
                            

url = "https://arxiv.org/pdf/2112.11598"
full_text = fitz_extract_pdf_text(url)

In [14]:
import requests
import PyPDF2

def pypdf_extract_pdf_text(pdf_url):
    try:
        # Download the PDF file from the URL
        # Sometimes the code just hangs, that's why
        # the 5 sec timeout has been included here.
        response = requests.get(pdf_url, timeout=5)
        response.raise_for_status()  # Raise an error for bad status codes
    except requests.exceptions.RequestException as e:
        return f"Error downloading PDF: {e}"

    pdf_path = "arxiv_paper.pdf"

    try:
        # Save the PDF file locally
        with open(pdf_path, "wb") as file:
            file.write(response.content)
    except IOError as e:
        return f"Error saving PDF: {e}"

    try:
        # Open and parse the PDF file using PyPDF2
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            full_text = ""

            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text = page.extract_text()
                if text:
                    full_text += f"Page {page_num + 1}:\n{text}\n\n"
                else:
                    full_text += f"Page {page_num + 1}:\n[No text found]\n\n"
    except (PyPDF2.errors.PdfReadError, IOError) as e:
        return f"Error reading PDF: {e}"

    return full_text

url = "https://arxiv.org/pdf/2112.11598"
full_text = pypdf_extract_pdf_text(url)
#print(full_text)

## System messages

In [15]:
comp_description = """

Competition Name: AI Mathematical Olympiad - Progress Prize 1

The goal of this competition is to create algorithms and models that can solve tricky math problems written in LaTeX format. Your participation will help to advance AI models’ mathematical reasoning skills and drive frontier knowledge.

The ability to reason mathematically is a critical milestone for AI. Mathematical reasoning is the foundation for solving many complex problems, from engineering marvels to intricate financial models. However, current AI capabilities are limited in this area.

The AI Mathematical Olympiad (AIMO) Prize is a new $10mn prize fund to spur the open development of AI models capable of performing as well as top human participants in the International Mathematical Olympiad (IMO).
This competition includes 110 problems similar to an intermediate-level high school math challenge. The Gemma 7B benchmark for these problems is 3/50 on the public and private test sets.

The assessment of AI models' mathematical reasoning skills faces a significant hurdle, the issue of train-test leakage. Models trained on Internet-scale datasets may inadvertently encounter test questions during training, skewing the evaluation process.

To address this challenge, this competition uses a dataset of 110 novel math problems, created by an international team of problem solvers, recognizing the need for a transparent and fair evaluation framework. The dataset encompasses a range of difficulty levels, from simple arithmetic to algebraic thinking and geometric reasoning. This will help to strengthen the benchmarks for assessing AI models' mathematical reasoning skills, without the risk of contamination from training data.

This competition offers an exciting opportunity to benchmark open AI models against each other and foster healthy competition and innovation in the field. By addressing this initial benchmarking problem, you will contribute to advancing AI capabilities and help to ensure that its potential benefits outweigh the risks.

Join us as we work towards a future where AI models’ mathematical reasoning skills are accurately and reliably assessed, driving progress and innovation across industries.
"""

In [16]:
chat_agent_system_message = f"""
You are a helpful Kaggle competition research assistant.
Competition: {comp_description}

1. You provide polite answers to simple questions.
If the user's input requires only a simple answer, then output your answer as JSON.

Example session:

Question: Hello. How are you?

You output:

{{
"Answer": "I'm fine, thanks.",
"Status": "DONE"
}}

2. You can also run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Output your response as a JSON string.

Your available actions are:

find_arxiv_research_papers:
e.g. find_arxiv_research_papers: [list of search keywords and phrases for a RAG search of the ArXiv database.]
Returns research papers from the ArXiv database.

run_web_search:
e.g. run_web_search: [list of search keywords and phrases for a web search]
Returns text content from search results.

You can only call one action at a time.

Example session:

Question: What are the latest techniques for detecting Pneumonia on x-rays using AI?
{{
"Thought": "I should look for relevant research papers in the ArXiv database by using find_arxiv_research_papers.",
"Action": {{"function":"find_arxiv_research_papers", "input": ["Pneumonia detection with AI", "Computer vision", "Object detection"]}},
"Status": "PAUSE"
}}

You will be called again with this:

Observation: <results>A list of research papers and their content</results>

You then output:
{{
"Answer": "Your final report.",
"Status": "DONE"
}}
""".strip()


# Set up the LLM

In [17]:
def make_llm_api_call(message):

    """
    Makes a call to the Llama3 model on Groq.
    Args:
        message_history (List of dicts): The message history
    Returns:
        response_text: (str): The text response from the LLM
    """

    response = chat.send_message(message)
    
    text_response = response.text

    return text_response


# Example

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction="You are a helpful assistant.",
)

chat = model.start_chat()

user_message = "What's your name?"

response = make_llm_api_call(user_message)

print(response)

I don't have a name. I am a large language model, and I am not a person. 😊 



## Set up the tools



### ArXiv RAG search tool

In [18]:
def run_faiss_search(query_text, top_k):
    
    # Run FAISS exhaustive search
    
    query = [query_text]

    # Vectorize the query string
    query_embedding = sent_model.encode(query)

    # Run the query
    # index_vals refers to the chunk_list index values
    scores, index_vals = faiss_index.search(query_embedding, top_k)
    
    # Get the list of index vals
    index_vals_list = index_vals[0]
    
    return index_vals_list
    

def run_rerank(index_vals_list, query_text):
    
    chunk_list = list(df_data['prepared_text'])

    # Replace the chunk index values with the corresponding strings
    pred_strings_list = [chunk_list[item] for item in index_vals_list]

    # Format the input for the cross encoder
    # The input to the cross_encoder is a list of lists
    # [[query_text, pred_text1], [query_text, pred_text2], ...]

    cross_input_list = []

    for item in pred_strings_list:

        new_list = [query_text, item]

        cross_input_list.append(new_list)


    # Put the pred text into a dataframe
    df = pd.DataFrame(cross_input_list, columns=['query_text', 'pred_text'])

    # Save the orginal index (i.e. df_data index values)
    df['original_index'] = index_vals_list

    # Now, score all retrieved passages using the cross_encoder
    cross_scores = cross_encoder.predict(cross_input_list)

    # Add the scores to the dataframe
    df['cross_scores'] = cross_scores

    # Sort the DataFrame in descending order based on the scores
    df_sorted = df.sort_values(by='cross_scores', ascending=False)
    
    # Reset the index (*This was missed previously*)
    df_sorted = df_sorted.reset_index(drop=True)

    pred_list = []

    for i in range(0,len(df_sorted)):

        text = df_sorted.loc[i, 'pred_text']

        # Get the arxiv id
        # original_index refers to the index values in df_filtered
        original_index = df_sorted.loc[i, 'original_index']
        arxiv_id = df_data.loc[original_index, 'id']
        cat_text = df_data.loc[original_index, 'cat_text']
        title = df_data.loc[original_index, 'title']

        # Crete the link to the research paper pdf
        link_to_pdf = f'https://arxiv.org/pdf/{arxiv_id}'
        

        # Extract the full text from the pdf document
        #full_text = pypdf_extract_pdf_text(link_to_pdf)


        item = {
            'arxiv_id': arxiv_id,
            'link_to_pdf': link_to_pdf,
            'cat_text': cat_text,
            'title': title,
            'abstract': text,
            #'full_text': full_text
        }

        pred_list.append(item)
    
    return pred_list


def print_search_results(pred_list, num_results_to_print):
    
    for i in range(0,num_results_to_print):
        
        pred_dict = pred_list[i]
        
        link_to_pdf = pred_dict['link_to_pdf']
        abstract = pred_dict['abstract']
        cat_text = pred_dict['cat_text']
        title = pred_dict['title']

        print('Title:',title)
        print('Categories:',cat_text)
        print('Abstract:',abstract)
        print('Link to pdf:',link_to_pdf)
        print()
    
   
def run_arxiv_search(query_text, top_k=10):
    
    # Run a faiss greedy search
    pred_index_list = run_faiss_search(query_text, top_k)

    # This returns a list of dicts with length equal to top_k
    pred_list = run_rerank(pred_index_list, query_text)
    
    # Print the results
    #print_search_results(pred_list, num_results_to_print)
    
    return pred_list
    

In [19]:
# Load the compressed array
embeddings = np.load(PATH_TO_EMBEDS)

# Access the array by the name you specified ('my_array' in this case)
embeddings = embeddings['array_data']

embeddings.shape

(2421966, 384)

In [20]:
# Load the compressed DataFrame

df_data = pd.read_csv(PATH_TO_DF, compression='gzip')

print(df_data.shape)

#df_data.head()

/tmp/ipykernel_33/3627369456.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(PATH_TO_DF, compression='gzip')


(2421966, 6)


In [21]:
# Initialize FAISS

import faiss

embed_length = embeddings.shape[1]

faiss_index = faiss.IndexFlatL2(embed_length)

# Add the embeddings to the index
faiss_index.add(embeddings)

faiss_index.is_trained

True

In [22]:
# Initialize sentence_transformers

from sentence_transformers import SentenceTransformer

sent_model = SentenceTransformer("all-MiniLM-L6-v2")

2024-06-18 06:29:31.258707: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 06:29:31.258827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 06:29:31.411291: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
# Initialize the cross_encoder for reranking

from sentence_transformers import CrossEncoder

# We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [24]:
# Example

query_text = """

I want to build an invisibility cloak like the one in Harry Potter.

"""


# RUN THE SEARCH
num_results_to_print = 20 # top_k = 300
pred_list = run_arxiv_search(query_text, top_k=5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
for i in range(0, len(pred_list)):
    
    print(i)

    url = pred_list[i]['link_to_pdf']

    full_text = pypdf_extract_pdf_text(url)

0
1
2
3
4


### Tavily web search

In [26]:
def run_tavily_search(query, num_results=10):

    """
    Uses the Tavily API to run a web search
    Args:
        query (str): The user query
        num_results (int): Num search results
    Returns:
        tav_response (json string): The search results in json format
    """

    # For basic search:
    tav_response = tavily_client.search(query=query, max_results=num_results)

    return tav_response


# Example

query = "How much does a bulldog weigh?"

results = run_tavily_search(query, num_results=2)

print(results)

{'query': 'How much does a bulldog weigh?', 'follow_up_questions': None, 'answer': None, 'images': None, 'results': [{'title': 'English Bulldog Growth & Weight Chart: Everything You Need To ... - Pawlicy', 'url': 'https://www.pawlicy.com/blog/english-bulldog-growth-and-weight-chart/', 'content': 'According to Care.com, puppies reach about 75% of their adult height at six months old. This will be around 10-13 inches tall for a male English Bulldog and approximately 9-11 inches tall for a female English Bulldog. As for weight, a 6-month-old male English Bulldog will weigh about 33 to 37 pounds, while a 6-month-old female English Bulldog ...', 'score': 0.94876, 'raw_content': None}, {'title': 'English Bulldog Growth and Weight Chart (Male & Female)', 'url': 'https://www.k9web.com/breeds/english-bulldog-growth-chart/', 'content': 'Male two-month-old Bulldogs will weigh between 9 and 12 pounds (4 and 5.4 kg), while females should weigh 7 and 10 pounds (3.1 and 4.5 kg). ... If your dog seems

# Set up the Agents

In [27]:
def run_chat_agent(message):

    print("---CHAT AGENT---")

    # Prompt the llm
    response = make_llm_api_call(message)

    print(response)

    return response



# Example

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()

message = "hello"

# Prompt the chat_agent
response = run_chat_agent(message)

response

---CHAT AGENT---
{
"Answer": "Hello! How can I help you today?",
"Status": "DONE"
} 



'{\n"Answer": "Hello! How can I help you today?",\n"Status": "DONE"\n} \n'

In [28]:
def run_router_agent(llm_response):

    """
    Route to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTER AGENT---")


    # Extract the status
    json_response = json.loads(llm_response)
    status = json_response['Status']
    #status = extract_json_str_value(response, "Status").strip()

    print("Status:", status)

    if status == 'PAUSE':
        print("Route: to_research_agent")
        return "to_research_agent"

    elif status == 'DONE':
        print("Route: to_final_answer")
        return "to_final_answer"



# Example

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()

message = "hello"

# Prompt the chat_agent
response = run_chat_agent(message)

# Run router_agent
route = run_router_agent(response)

route

---CHAT AGENT---
{
"Answer": "Hello!",
"Status": "DONE"
} 

---ROUTER AGENT---
Status: DONE
Route: to_final_answer


'to_final_answer'

In [34]:
def run_research_agent(llm_response):

    print("---RESEARCH AGENT---")

    # Extract the status
    json_response = json.loads(llm_response)
    action_dict = json_response['Action']
    func_to_run = action_dict['function']
    func_input_list = action_dict['input']
    
    answer_list = []
    final_answer_list = []

    if func_to_run == "find_arxiv_research_papers":
        for search_query in func_input_list:
            answer_list = run_arxiv_search(search_query, top_k=NUM_SEARCH_RESULTS)
            #answer_list.append(answer)
            # Concat the lists
            final_answer_list = final_answer_list + answer_list
            
        # Read the papers here
        # We do that here because the colde will not
        # wait for the web request to return before going on.
        print('Extracting text from pdf papers...')
        print("Num files:", len(final_answer_list))
        for i in range(0, len(final_answer_list)):
            
            print(i, end='\r')
            
            # Get the pdf url
            url =  final_answer_list[i]['link_to_pdf']
            
            # Extract the full text of the pdf
            full_text = pypdf_extract_pdf_text(url)
            
            # Add a new key and value to the dictionary
            final_answer_list[i]['full_text'] = full_text
        
    else:
        for search_query in func_input_list:
            answer = run_tavily_search(search_query, num_results=NUM_SEARCH_RESULTS)
            final_answer_list.append(answer)

    print("func_to_run:", func_to_run)
    print("func_arg:", func_input_list)
    #print("Output:", answer_list)
    print("Search complete.")

    return final_answer_list



# Example

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()

message = "Hello"


# Prompt the chat_agent
response = run_chat_agent(message)

# Run router_agent
route = run_router_agent(response)


if route == "to_research_agent":
    answer = run_research_agent(response)

    # Update message history
    #message = {"role": "user", "content": f"Observation: {answer}"}
    #message_history.append(message)



---CHAT AGENT---
{
"Answer": "Hello! 👋 How can I help you with your AI Mathematical Olympiad journey?",
"Status": "DONE"
} 

---ROUTER AGENT---
Status: DONE
Route: to_final_answer


In [30]:
def run_final_answer_agent(llm_response):

    print("---FINAL ANSWER AGENT---")

    json_response = json.loads(llm_response)
    final_answer = json_response['Answer']

    print("Final answer:", final_answer)

# Run the system

In [35]:
model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()


user_input = "What is the current state of the art techniques for using large language models to solve math problems?"

for i in range(0,10):

    # Prompt the chat_agent
    llm_response = run_chat_agent(user_input)

    # Run router_agent
    route = run_router_agent(llm_response)


    if route == "to_research_agent":

        answer = run_research_agent(llm_response)
        
        user_input = f"Observation: {answer}"

    else:

        run_final_answer_agent(llm_response)

        break



---CHAT AGENT---
{
"Thought": "I should look for relevant research papers in the ArXiv database by using find_arxiv_research_papers.",
"Action": {"function":"find_arxiv_research_papers", "input": ["Large language models", "Math problem solving", "AI", "Mathematical reasoning"]},
"Status": "PAUSE"
} 

---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent
---RESEARCH AGENT---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting text from pdf papers...
Num files: 12
func_to_run: find_arxiv_research_papers
func_arg: ['Large language models', 'Math problem solving', 'AI', 'Mathematical reasoning']
Search complete.
---CHAT AGENT---
{
"Answer": "Large language models (LLMs) are showing promising results in solving math problems, but the current state-of-the-art techniques are still under development. Some key approaches include:\n\n* **Symbolic Reasoning:**  LLMs can be trained to understand and manipulate mathematical expressions written in LaTeX format.  This involves learning rules of logic and algebra, and applying them to solve problems. \n* **Code Generation:** LLMs can be trained to generate code in programming languages like Python that can solve math problems. This leverages the ability of LLMs to understand and generate code. \n* **Problem Decomposition:** LLMs can be trained to break down complex math problems into smaller, more manageable sub-problems. This allows them to tackle challenging 

In [37]:
len(answer)

12

# Run infinitely with a user input field

In [39]:
model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()


while True:

    print()
    print("==========")
    user_input = input("Enter something (or 'q' to quit): ")
    print("==========")
    
    i = i + 1

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break  # Exit the loop


    for i in range(0,10):

        # Prompt the chat_agent
        llm_response = run_chat_agent(user_input)

        # Run router_agent
        route = run_router_agent(llm_response)


        if route == "to_research_agent":

            answer = run_research_agent(llm_response)
            
            user_input = f"Observation: {answer}"

        else:

            run_final_answer_agent(llm_response)

            break



Enter something (or 'q' to quit):  I want to build an invisibility cloak like the one harry Potter has. Are their any resources that I can refer to?


---CHAT AGENT---
{
"Answer": "Unfortunately, invisibility cloaks like the one in Harry Potter are purely fictional and not possible with current technology.  However, there are some fascinating real-world research projects exploring the possibilities of light manipulation and cloaking, though these are not close to creating actual invisibility.  If you're interested in exploring these fields, you could research topics like 'Metamaterials', 'Light Bending', and 'Optical Illusion'.",
"Status": "DONE"
} 

---ROUTER AGENT---
Status: DONE
Route: to_final_answer
---FINAL ANSWER AGENT---
Final answer: Unfortunately, invisibility cloaks like the one in Harry Potter are purely fictional and not possible with current technology.  However, there are some fascinating real-world research projects exploring the possibilities of light manipulation and cloaking, though these are not close to creating actual invisibility.  If you're interested in exploring these fields, you could research topics like '

Enter something (or 'q' to quit):  Please research these and other realted ares


---CHAT AGENT---
{
"Thought": "I need to gather information about metamaterials, light bending, optical illusions, and other related areas to provide a comprehensive overview of real-world invisibility research.",
"Action": {"function": "run_web_search", "input": ["metamaterials invisibility cloak", "light bending technology", "optical illusion research", "invisibility research projects"]},
"Status": "PAUSE"
} 

---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent
---RESEARCH AGENT---
func_to_run: run_web_search
func_arg: ['metamaterials invisibility cloak', 'light bending technology', 'optical illusion research', 'invisibility research projects']
Search complete.
---CHAT AGENT---
{
"Answer": "While a Harry Potter-style invisibility cloak is still firmly in the realm of fantasy, research into metamaterials, light bending, and optical illusions is making significant strides. \n\nMetamaterials are engineered materials with properties not found in nature. They can manipulate light in

Enter something (or 'q' to quit):  Please search academic papers with the keywords "harry potter invisibility"


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting text from pdf papers...
Num files: 24
func_to_run: find_arxiv_research_papers
func_arg: ['Harry Potter', 'cultural studies', 'education', 'literature', 'fantasy', "children's literature", 'magic', 'wizarding world']
Search complete.
---CHAT AGENT---
{
"Answer": "The search for academic papers using only 'Harry Potter' resulted in a mix of interesting findings.  There are a number of papers exploring various aspects of the series, including:\n\n* **Computational Linguistics:**  Papers analyzing the use of language in the Harry Potter books, such as the paper \"Harry Potter and the Action Prediction Challenge from Natural Language,\" which focuses on predicting spells based on text.
* **Economics:**  There are papers analyzing the economic system of the wizarding world, such as \"Potterian Economics,\" which explores the Potterian economic model and compares it to professional economic models.
* **Education:** Papers exploring the use of the Harry Potter series in teaching var

JSONDecodeError: Invalid control character at: line 2 column 438 (char 439)